In [12]:
import re
import requests

In [13]:
current = ""
fin = open("kingjames.txt") # file downloaded from http://www.gutenberg.org/1/10/
for line in fin.readlines():
        current += line
print (len(current))
fin.close()
print (type(current))


4351846
<class 'str'>


In [14]:
#start here
#"*** START OF THIS PROJECT GUTENBERG EBOOK THE KING JAMES BIBLE ***"

#end here
#"End of the Project Gutenberg EBook of The King James Bible"

In [15]:
#remove the informational text at the beginning 
currentB = current.split("*** START OF THIS PROJECT GUTENBERG EBOOK THE KING JAMES BIBLE ***", 1)[1]
print (len(currentB))

4351306


In [16]:
#remove the copyright and disclaimers text at the end
currentE = currentB.rsplit("End of the Project Gutenberg EBook of The King James Bible", 1)[0]
print (len(currentE))

4332586


In [17]:
currentT = currentE.split("***")
print (len(currentT)) #this is correct, as we have two testaments in the text file divided by the *** characters.

2


In [18]:
newTest = currentT[1]
oldTest = currentT[0]
oldT = oldTest.split("\nANOTHER BOOK STARTS HERE")
newT = newTest.split("\nANOTHER BOOK STARTS HERE")
print (len(oldT))
print (len(newT))

1
1


In [19]:
booksNames = ["Genesis","Exodus","Leviticus","Numbers","Deuteronomy","Joshua","Judges","Ruth","1 Samuel","2 Samuel",
           "1 Kings","2 Kings","1 Chronicles","2 Chronicles","Ezra","Nehemiah","Esther","Job","Psalms","Proverbs",
           "Ecclesiastes","Song of Solomon","Isaiah","Jeremiah","Lamentations","Ezekiel","Daniel","Hosea",
           "Joel","Amos","Obadiah","Jonah","Micah","Nahum","Habakkuk","Zephaniah","Haggai","Zechariah","Malachi", 
            "Matthew","Mark","Luke","John","Acts","Romans","1 Corinthians","2 Corinthians","Galatians","Ephesians",
             "Philippians","Colossians","1 Thessalonians","2 Thessalonians","1 Timothy","2 Timothy","Titus",
            "Philemon","Hebrews","James","1 Peter","2 Peter","1 John","2 John","3 John","Jude","Revelation"]

booksAbbr = ["gen", "exo", "lev", "num", "deu", "jos", "jdg", "rut", "1sa", "2sa", "1ki", "2ki", "1ch", "2ch", "ezr", 
             "neh", "est", "job", "psa", "pro", "ecc", "sng", "isa", "jer", "lam", "ezk", "dan", "hos", "jol", "amo", 
             "oba", "jon", "mic", "nam", "hab", "zep", "hag", "zec", "mal", "mat", "mrk", "luk", "jhn", "act", "rom",
             "1co", "2co", "gal", "eph", "php", "col", "1th", "2th", "1ti", "2ti", "tit", "phm", "heb", "jas", "1pe",
             "2pe", "1jn", "2jn", "3jn", "jud", "rev"]

oldTBooks, newTBooks = booksNames[:39], booksNames[39:]

In [20]:
abbreviations = {}
for idx in range(len(booksNames)):
    abbreviations[booksNames[idx]] = booksAbbr[idx]

In [21]:
fullDict = {}
for i in range(len(oldTBooks)):
    fullDict[oldTBooks[i]] = oldT[i]
for i in range(len(newTBooks)):
    fullDict[newTBooks[i]] = newT[i]

IndexError: list index out of range

In [67]:
count, ct = 0, 0
versesDict = {}
for key, value in fullDict.items():
    verses = re.split(r'(\d+:\d+)', value)
    lst = []
    for k in range(1, len(verses), 2):
        ct += 1
        newDict = {}
        newLst = []
        chapterVerseLst = verses[k].split(":")
        ch, ve = chapterVerseLst[0], chapterVerseLst[1]
        
        url = "https://www.bible.com/en-GB/bible/351/" + abbreviations[key] + "." + ch + "." + ve + ".bysb"
        r = requests.get(url, auth=('user', 'pass'))
        verseKinya = ""
        
        if 'class="content">' in r.text:
            verseKinya = r.text.rsplit('class="content">', 1)[1].split('</span></span>')[0]
        else:
            count += 1
            
        newLst.append(verses[k+1].replace("\n"," "))
        newLst.append(verseKinya)
        newDict[verses[k]] = newLst
        lst.append(newDict)
        
    versesDict[key] = lst
    
print(count, ct)

2595 31102


In [68]:
kk = 0
for i in versesDict:
    kk += 1
print (kk)

66


In [69]:
print (versesDict["Acts"][0])

{'1:1': [' The former treatise have I made, O Theophilus, of all that Jesus began both to do and teach, ', 'Muri cya gitabo cya mbere nanditse ibyo Yesu yabanje gukora no kwigisha byose, ']}


In [70]:
kinya2engl = ""
for i in versesDict:
    for k in versesDict[i]:
        for key, value in k.items():            
            kinya2engl += value[1]
            kinya2engl += " ||| "
            kinya2engl += value[0]
            kinya2engl += "\n\n"



t = open("kinyatest.txt", 'w')
t.write(kinya2engl)
t.close()  